In [3]:
from datetime import datetime
from collections import defaultdict
import calendar
from pprint import pprint

def parse_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d")

def get_month_range(year_month):
    year, month = map(int, year_month.split("-"))
    start = datetime(year, month, 1)
    end_day = calendar.monthrange(year, month)[1]
    end = datetime(year, month, end_day)
    return start, end

def get_overlap(start1, end1, start2, end2):
    start = max(start1, start2)
    end = min(end1, end2)
    if start <= end:
        return start, end
    return None, None

def generate_monthly_bill(item_list, target_month):
    month_start, month_end = get_month_range(target_month)
    days_in_month = (month_end - month_start).days + 1

    grouped_data = defaultdict(lambda: {"qty": 0, "amount": 0.0})

    for item in item_list:
        item_start = parse_date(item["start_date"])
        item_end = parse_date(item["end_date"])  # using end_date now

        active_start, active_end = get_overlap(item_start, item_end, month_start, month_end)
        if not active_start:
            continue

        active_days = (active_end - active_start).days + 1
        rate = float(item["rate"])
        qty = int(item["qty"])

        prorated_amount = round((rate * qty) * (active_days / days_in_month), 2)
        billing_period = f"{active_start.strftime('%Y-%m-%d')} to {active_end.strftime('%Y-%m-%d')}"

        key = (item["item_code"], rate, billing_period)
        grouped_data[key]["qty"] += qty
        grouped_data[key]["amount"] += prorated_amount

    total_revenue = 0.0
    line_items = []

    for (item_code, rate, period), data in grouped_data.items():
        amount = round(data["amount"], 2)
        line_items.append({
            "item_code": item_code,
            "rate": rate,
            "qty": data["qty"],
            "amount": amount,
            "billing_period": period
        })
        total_revenue += amount

    return {
        "line_items": line_items,
        "total_revenue": round(total_revenue, 2)
    }

# input data
item_list = [
    {"item_code": "A101", "start_date": "2024-10-25", "end_date": "2024-11-10", "qty": 5, "rate": 100.0},
    {"item_code": "A101", "start_date": "2024-11-11", "end_date": "2024-11-25", "qty": 3, "rate": 100.0},
    {"item_code": "A102", "start_date": "2024-11-01", "end_date": "2024-11-30", "qty": 2, "rate": 200.0}
]


pprint(generate_monthly_bill(item_list, "2024-11"))


{'line_items': [{'amount': 166.67,
                 'billing_period': '2024-11-01 to 2024-11-10',
                 'item_code': 'A101',
                 'qty': 5,
                 'rate': 100.0},
                {'amount': 150.0,
                 'billing_period': '2024-11-11 to 2024-11-25',
                 'item_code': 'A101',
                 'qty': 3,
                 'rate': 100.0},
                {'amount': 400.0,
                 'billing_period': '2024-11-01 to 2024-11-30',
                 'item_code': 'A102',
                 'qty': 2,
                 'rate': 200.0}],
 'total_revenue': 716.67}
